# DataDataData

Here's a sample class wrapping the main abstraction, the `Dataset`.

In [1]:
import csv
import random


class Dataset(object):
    """ Encapsulates unlabelled and labelled examples. """
    def __init__(self, text_to_label=None):
        self.text_to_label = text_to_label or {}
        
    def add_label(self, text, label):
        self.text_to_label[text] = label
        
    def iter_texts(self, accept=lambda label: True):
        return ((text, label) for text, label in 
                 self.text_to_label.items() if accept(label))
    
    def to_csv(self, fname):
        with open(fname, 'w') as f:
            w = csv.writer(f, delimiter=',')
            for text, label in self.text_to_label.items():
                w.writerow((label, text))
    
    @classmethod
    def from_csv(cls, fname):
        with open(fname) as f:
            return cls({text: label for text, label in
                        csv.reader(f, delimiter=',')})
        
    def update(self, other):
        self.text_to_label.update(other.text_to_label)

# 20 newgroups

* Take the training set.
* Remove every second label.

In [2]:
from sklearn.datasets import fetch_20newsgroups

def build_newsgroups(subset='train'):
    """ Fetches newsgroupd data and returns a Dataset. """
    newsgroups_train = fetch_20newsgroups(subset=subset)
    label_names = {index: name for index, name in 
                   enumerate(newsgroups_train.target_names)}
    return Dataset({text: label_names[index] 
                    for text, index in zip(newsgroups_train.data, 
                                           newsgroups_train.target)})

def unlabel(dataset, p=0.5):
    """ Randomly removes some labels. """
    for text, label in dataset.iter_texts():
        if random.random() > p:
            dataset.add_label(text, None)


train = build_newsgroups()
unlabel(train)

# Active sampling with classifier

Here is a straw man active sampler that:
* trains a classifier on the labelled data
* predicts the labels of unlabelled data
* selects text with a specific label profile

In [3]:
from collections import Counter

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline


def iter_active_sample(dataset, accept=lambda i: True, limit=None, cross_validate=True):
    """ Trains a model from the labelled data.
    Yields unlabelled (but predicted examples that evaluate True in accept()).
    """
    X, y = [], []
    for text, label in list(dataset.iter_texts(lambda l: l is not None)):
        X.append(text)
        y.append(label)
    print('Got {} labelled samples'.format(len(X)))
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('clf', SGDClassifier(loss='log')),
    ])
    if cross_validate:
        print('Cross-validating')
        scores = cross_val_score(pipeline, X, y, cv=3)
        print("Cross-validated accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print('Refitting')
    pipeline.fit(X, y)
    X, y = [], []
    for text, label in list(dataset.iter_texts(lambda l: l is None)):
        X.append(text)
        y.append(label)
    print('Predicting {} unlabelled'.format(len(X)))
    yielded = 0
    for probs, text in zip(pipeline.predict_proba(X), X):
        predictions = dict(zip(pipeline.classes_, probs))
        if accept(predictions):
            yield text, predictions
            yielded += 1
            if limit and yielded == limit:
                break
    

def iter_random_unlabelled(dataset, limit=None):
    unlabelled = list(dataset.iter_texts(accept=lambda label: label is None))
    random.shuffle(unlabelled)
    yielded = 0
    for text, label in unlabelled:
        yield text, label
        yielded += 1
        if limit and yielded == limit:
            break

In [4]:
for text, preds in iter_active_sample(train, 
                                      accept=lambda preds: max(preds.values()) < 0.7,
                                      limit=5,
                                      cross_validate=True):
    print('Labels\t{}\nText\t{}\n'.format(
        list(sorted(preds.items(), key=lambda i: i[1], reverse=True))[:3],
        repr(text)
    ))

Got 5656 labelled samples
Cross-validating


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will b

Cross-validated accuracy: 0.73 (+/- 0.02)
Refitting


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Predicting 5658 unlabelled
Labels	[('comp.os.ms-windows.misc', 0.49999519650775315), ('soc.religion.christian', 0.49998825435273836), ('comp.windows.x', 1.6549139380699382e-05)]
Text	'From: yuting@Xenon.Stanford.EDU (Eugene Y. Kuo)\nSubject: Any updated Canon BJ-200 driver?\nOrganization: Computer Science Department, Stanford University.\nLines: 8\n\nHi ... can anyone tell me where I can get a copy of updated Canon BJ-200\nprinter driver for Windows 3.1, if any ? I have ver 1.0 which comes with\nmy BJ-200 printer, I just wonder if there is any newer version.\n\nThanks very much, please email.\n\n\n\n'

Labels	[('rec.autos', 0.5), ('talk.politics.guns', 0.5), ('rec.sport.baseball', 1.4280381443855174e-27)]
Text	'From: callison@uokmax.ecn.uoknor.edu (James P. Callison)\nSubject: Re: WARNING.....(please read)...\nNntp-Posting-Host: uokmax.ecn.uoknor.edu\nOrganization: Engineering Computer Network, University of Oklahoma, Norman, OK, USA\nLines: 32\n\nIn article <1993Apr17.051701.3419@nusc

/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/base.py:347: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


In [5]:
for text, label in iter_random_unlabelled(train, limit=3):
    print('Label\t{}\nText\t{}\n'.format(
        label,
        repr(text)
    ))

Label	None
Text	'From: ccdarg@dct.ac.uk (Alan Greig)\nSubject: Re: ATF BURNS DIVIDIAN RANCH! NO SURVIVORS!!!\nOrganization: Dundee Institute of Technology\nLines: 22\n\nIn article <C5tEnu.112F@ns1.nodak.edu>, green@plains.NoDak.edu (Bill Green) writes:\n\n> And a few other questions.  Like I said, I believe the actions taken, in\n> general, were proper.  But I still have some reservations.\n\nWe\'ve heard a lot of talk about brainwashing in Waco but the brainwashing\nof the general population never ceases to amaze me. Here is an\nexample of action being taken which results in the worst possible\noutcome and despite people\'s deep intuition telling them something\nis wrong the programming will still cut in and say that the\nagents probably acted in good faith. NO THEY DIDN\'T. They either did\nnot have enough information to act in good faith or else they acted\nknowing the risk. Sums up human stupidity all over and one of these\ndays it will destroy the fucking planet: "Oh sorry. Didn\'

# Data programming

We can weakly supervise using precise functions. Note that we have no fancy model above the labelling functions.

In [6]:
from collections import Counter


def hockey_puck(text):
    if 'puck' in text.lower():
        return 'rec.sport.hockey'


def baseball_home_run(text):
    if 'home run' in text.lower():
        return 'rec.sport.baseball'
    

def hockey_NHL(text):
    if not 'nhl' in text.lower():
        return '!rec.sport.hockey'


def iter_apply_funcs(data, funcs):
    for text, _ in data:
        votes = Counter(func(text) for func in funcs).most_common(1)
        if votes[0][0] is None:
            continue
        yield text, votes

In [7]:
for text, votes in iter_apply_funcs(iter_random_unlabelled(train, limit=250), 
                                    [hockey_NHL, hockey_puck, baseball_home_run]):
    print('Label\t{}\nText\t{}\n'.format(votes, repr(text)))

Label	[('!rec.sport.hockey', 1)]
Text	"From: etxonss@ufsa.ericsson.se (Staffan Axelsson)\nSubject: WC 93: Results, April 20\nOrganization: Ericsson Telecom, Stockholm, Sweden\nLines: 162\nNntp-Posting-Host: uipc104.ericsson.se\n\n\n 1993 World Championships in Germany:\n ====================================\n\n Group A results:\n\n SWEDEN - CANADA  1-4 (0-0,1-1,0-3)\n\n 1st:\n 2nd: CAN 0-1 Geoff Sanderson      (Kevin Dineen)                  7:24\n      SWE 1-1 Patrik Juhlin        (Jan Larsson)                  15:23 (pp)\n 3rd: CAN 1-2 Geoff Sanderson                                      5:54 (ps)\n      CAN 1-3 Mike Gartner         (Greg Johnson,Adam Graves)     10:44 \n      CAN 1-4 Rod Brind'Amour      (Shayne Corson)                19:59\n\n            Shots on goal:    Penalties:    Attendance:     Referee:\n Sweden     10 15 12 - 37     4*2min        6,500           Rob Hearn (USA)\n Canada     10 13  6 - 29     6*2min\n\n Bill Ranford stopped 36 shots to lead Canada to a 4-1 v

# Annotation tool


In [8]:
from IPython.display import clear_output, display
import ipywidgets as widgets
import functools

In [9]:
class AnnotationPane(object):
    def __init__(self, queue, label_scheme):
        self.text_pane = widgets.HTML()
        self.buttons = []
        for desc, label in label_scheme:
            button = widgets.Button(
                description=desc
            )
            button.on_click(functools.partial(self.on_click, label))
            self.buttons.append(button)
        
        self.view = widgets.VBox([
            widgets.HBox(self.buttons),
            self.text_pane,  
        ])
        self.queue = list(queue)
        self.draw()
        display(self.view)
        self.responses = []

    def on_click(self, label, button):
        self.responses.append((text, label))
        self.draw()
        
    def draw(self):
        if not self.queue:
            for button in self.buttons:
                button.disabled = True
            self.text_pane.value = '<p>Finished</p>'
        else:
            self.text = self.queue.pop(0)
            self.text_pane.value = '<p>{}</p>'.format(self.text)

In [10]:
pane = AnnotationPane(list(iter_random_unlabelled(train, limit=10)),
                      [('Yes', True),
                       ('No', False)])

VBox(children=(HBox(children=(Button(description='Yes', style=ButtonStyle()), Button(description='No', style=ButtonStyle()))), HTML(value='<p>("From: bcwhite@sunee.uwaterloo.ca (Brian C. White)\\nSubject: Re: SCSI on dos\\nKeywords: SCSI, DOS, streamer\\nOrganization: University of Waterloo\\nLines: 15\\n\\nIn article <1993Apr19.132748.18044@julian.uwo.ca> wlsmith@valve.heart.rri.uwo.ca (Wayne Smith) writes:\\n>It was the Seagate 296N and the ST-02 controller.  I found that the\\n>controller couldn\'t keep up with a 1:1 interleave, so the best I could do\\n>with the drive was a 2:1 interleave and a data transfer of about 450 k/sec.\\n\\nAccording to what others have told me, the ST-296N is difficult to run at\\nthe 1:1 interleave even though Seagate claims it.  I have a non-pc system\\n(don\'t ask what it is, you probably haven\'t heard of it) that is built\\naround SCSI and it can\'t do 1:1, either.\\n\\n                                        Brian\\n                           ( bcwhite@sunee.uwaterloo.ca )\\n\\n-------------------------------------------------------------------------------\\n    In theory, theory and practice are the same.  In practice, they\'re not.\\n", None)</p>')))